<a href="https://colab.research.google.com/github/POOJA-BFRS01535/chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Ecommerce Chatbot

### Problem Statement

Design a chatbot which can be used by any D2C ecommerce company to answer first level most common queries of their customers. If chatbot is unable to support it will then only pass the customer to manual customer support. This will help in reducing the no of support agents required to answer user questions plus customers will get a quicker response without having to wait for replies for basic questions.

**Use** **cases to solve**


1.   Order related questions
2.   Product information
3. Return and Refund queries
4. Shipment status



# Version 1

### Chatbot for post order journey ecommerce sellers
* `initialise_conversation()`
Identify intent of each user message. If not clear ask further questions. If still not clear redirect to manual agent. `identify_intent()`
* `check_intent_supported()`
Once intent is identified if intent is from list of supported intent then take the flow as per that intent else redirect to manual agent.

* For get order details intent

  * get order id from user - `get_order_id`
  * once obtained - pull out order info from db/shopify API and make it json. - `extract_order_info`
  * Based on what user asked prepare response - `prepare_order_response`
* For get product details
  * get ecommerce site url from settings for example phool.co
  * first check what the user has asked. Form a seach query for it. `create_search_query`
  * Then go to website of the ecommerce company and search for the product in it. `perform_search`
  * then formulate the results and respond back - `respond_to_user_query`
* For return and refund info
  * get details of return and refund policy of company from urls of the pages in their websites in settings - `prepare_refund_return_policy`
  * answer user query based on the return and refund policy - `return_return_refund_policy_info`
* Get_shipment_info
  * get shipment awb from user - `get_shipment_awb`
  * once obtained - pull out order info from shiprocket API and pull relevant info from it. - `get_shipment_info`
  * Based on what user asked prepare response - `prepare_shipment_response`



###  Problems Faced
  As I started working on this approach I got confused with 2 questions.
  * How do I manage if user changes intent mid conversation? How will the system design look for it?
  * How do I continue the conversation after I have switched to the another flow for like get order details etc. As I want chatbot to pick up conservation from where it left so that it looks like human like interaction.

  Both these questions made it difficult to design the system. After attending the part 2 of shop assist I decided to for now make assumptions that the chatbot will not encounter above scenarios. User will not change intent and only one intent will be handled at a time.


In [60]:
# Install OpenAI library
!pip install -U -q openai tenacity

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/upgrad/GenAI_Course_Master/Chatbot_Project/')
!ls

# Import the libraries
import pandas as pd
from IPython.display import display, HTML
# Set the display width to control the output width
pd.set_option('display.width', 100)
# Read the dataset and read the Laptop Dataset



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
OPENAI_API_Key.txt  products.csv


In [61]:
# Import the libraries
import os, json, ast
import requests
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt

# read the OpenAI API key
openai.api_key = open("OPENAI_API_Key.txt", "r").read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key


In [62]:
# INTENTS = {
#     "order_status": ["order status", "track my order", "where is my order", "order information", "details of order", "order confirmation", "order history"],
#     "product_info": ["product features", "product details", "product info"],
#     "return_policy": ["return policy", "how to return", "refund policy"],
#     "shipping_info": ["shipping details", "delivery time", "shipping info"],
# }

INTENTS = {
    "order_status": [
        "order status", "order information", "order details", "order update", "order number",
        "details of order", "order confirmation", "order history", "check my order",
        "shipping details", "delivery time", "shipping info", "shipment status",
        "where is my shipment", "track my shipment", "delivery status", "shipping status",
        "when will my order arrive", "delivery tracking", "shipping update",
        "estimated delivery", "shipment tracking",  "order tracking", "order not received",
        "track my order", "where is my order"
    ],
    "product_info": [
        "product features", "product details", "product info", "product specifications",
        "product availability", "product compatibility", "product description",
        "product price", "product reviews", "product warranty", "compare products",
        "product colors", "product sizes"
    ],
    "return_policy": [
        "return policy", "how to return", "refund policy", "return process",
        "return item", "return instructions", "exchange policy", "refund process",
        "how to exchange", "return guidelines", "refund request", "return eligibility"
    ],
}


ECOM_WEBSITE = "https://phool.co/"

REFUND_URL = "https://phool.co/pages/terms-of-use"

In [63]:
print(INTENTS.keys())

products_df = pd.read_csv('products.csv')

products_df.head()


dict_keys(['order_status', 'product_info', 'return_policy'])


,Product Name,Price (INR),URL,Description
0,Phool Badrinath Kesar Chandan Incense Cones,185,https://phool.co/products/badrinath-kesar-chan...,Experience the divine fragrance of Badrinath K...
1,Phool Badrinath Kesar Chandan Bambooless Incen...,265,https://phool.co/products/badrinath-kesar-chan...,"These bambooless incense sticks offer a pure, ..."
2,Phool Ayodhya Soumya Chandan Bambooless Incens...,265,https://phool.co/products/ayodhya-soumya-chand...,"Infused with the calming scent of chandan, the..."
3,Phool Natural Incense Sticks - Oudh,245,https://phool.co/products/natural-incense-stic...,"Enjoy the rich, woody aroma of oudh with these..."
4,Phool Natural Incense Sticks - Lavender,165,https://phool.co/products/natural-incense-stic...,These lavender incense sticks provide a soothi...


In [64]:


def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"

    system_message = """
    You are an experienced customer support agent for an ecommerce direct to customer brand and your goal is to identify the intent of incoming customer queries.
    You will be handling 3 types of user queries. Check user order, check status of shipment, get product info, answer questions about return and refund policy.
    In case user asks questions which you do not know how to handle redirect user to manual customer support.

    You need to ask relevant questions and understand the user intent first and reply back with what is user intent out of below intents.
    If you are unable to identify intent continue the conversation till to find the intent.
    In below dictionary, keys are intents and the list of values for each key represent some sample input which user can give for that intent.
    Based on these sample inputs identify the intent of user.
    {delimiter}
    {INTENTS}
    {delimiter}

    Your output should be a single word intent out of {INTENTS.keys()}. If you are unable to identify indent then continue conversation with user.

    {delimiter}


    ### Examples:

Example 1:

Customer Query: "Can you tell me where my package is right now?"
Response: "order_status"

Example 2:

Customer Query: "I need information on how to return a product I purchased last week."
Response: "return_policy"

Example 3:

Customer Query: "What are the features of the new smartphone you have on sale?"
Response: "product_info"

Example 4:

Customer Query: "I haven't received an order confirmation email. Can you check my order?"
Response: "order_status"


Example 5:

Customer Query: "Hi"
Response: "Hi! How can I help you?"
Customer Follow-Up: "It's about the delivery of my order 233."
Response: "order_status"


Example 6:

Customer Query: "I need assistance."
Response: "Hello! I'm here to assist you. Can you please tell me if want to know more about our products, or some existing order. I can also help you with our return and refund policy."
Customer Follow-Up: "nothing"
Response: "Sorry I could not understand your question. Can you please tell me if want to know more about our products, or some existing order. I can also help you with our return and refund policy."

  """

    conversation = [{"role": "system", "content": system_message}]
    return conversation



In [65]:
# Define a Chat Completions API call
# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions(input, json_format = False):
    MODEL = 'gpt-3.5-turbo'

    system_message_json_output = """<<. Return output in JSON format to the key output.>>"""

    # If the output is required to be in JSON format
    if json_format == True:
        # Append the input prompt to include JSON response as specified by OpenAI
        input[0]['content'] += system_message_json_output

        # JSON return type specified
        chat_completion_json = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            response_format = { "type": "json_object"},
            seed = 9877)

        output = json.loads(chat_completion_json.choices[0].message.content)

    # No JSON return type specified
    else:
        chat_completion = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            seed = 9866)

        output = chat_completion.choices[0].message.content

    return output

In [66]:
# system_msg = initialize_conversation()
# debug_conversation = system_msg
# print(system_msg[0]["content"])

In [67]:
# user_input1 = "Hi"
# debug_conversation.append({'role':'user','content':user_input1})

# debug_response_assistant = get_chat_completions(debug_conversation) ## Chat Completions API


In [68]:
# debug_conversation.append(({"role": "assistant", "content": debug_response_assistant}))


In [69]:
# debug_conversation

In [70]:
# user_input2 = "I want to know more about refund policy"
# last_user_msg = user_input2
# debug_conversation.append({'role':'user','content':user_input2})

# debug_response_assistant = get_chat_completions(debug_conversation) ## Chat Completions API
# print(debug_response_assistant)



In [71]:
def check_intent_identified(assistant_response):
  if assistant_response in INTENTS.keys():
    return True



In [72]:
# def get_order_id(last_user_input):
#   delimiter = "####"
#   confirm_order_id_msgs = [
#       {'role':'system','content':f"""
#         You are a customer support executive. Need your help to check if user has passed order id in the input in the message.
#         Reply back with a json having 2 keys.
#         'order_id_present' which will either be 'true' or 'false' based on whether order is present,
#         and order_id which will contain order_id when present and null when not.
#         {delimiter}"""},
#       {'role': 'user', 'content': f"Here is the message {last_user_input}"}
#       ]
#   response = openai.chat.completions.create(
#       model="gpt-3.5-turbo",
#       messages = confirm_order_id_msgs,
#       response_format={ "type": "json_object" },
#       seed = 1234
#       )
#   json_output = json.loads(response.choices[0].message.content)

#   if json_output.get('order_id_present'):
#     return json_output.get('order_id')


In [73]:
def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai.moderations.create(input=user_input)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

In [74]:
def get_order_details(order_id):
  # print("inside get order details")
  # return {
  #     'order_id': 122,
  #     'total_amount': 299.99,
  #     'delivery_status': 'Shipped',
  #     'current_location': 'Warehouse B',
  #     'expected_delivery_date': '2023-01-18',
  #     'customer_name': 'Alice Brown',
  #     'product_id': 4,
  #     'product_name': 'Tablet'
  #     }

  # https://apiv2.shiprocket.in/v1/tracking/19041580387436
  url = f"https://apiv2.shiprocket.in/v1/tracking/{order_id}"  # Replace with your actual endpoint
  response = requests.get(url)

  if response.status_code == 200:
      return response.json()
  else:
      return f"Error: Unable to fetch order details. Status code: {response.status_code}"


In [75]:
# Function to search product details
def search_product(query):
  results = products_df[products_df.apply(lambda row: query.lower() in row['Product Name'].lower() or query.lower() in row['Description'].lower(), axis=1)]
  product_info = "\n".join([f"Title: {row['Product Name']}, Price: {row['Price (INR)']}, URL: {row['URL']}, Description: {row['Description']}" for index, row in results.iterrows()])
  return product_info



In [76]:

available_functions = {
    "get_order_details": get_order_details,
    "search_product": search_product
}

def execute_function_call(function_name, arguments):
    function = available_functions.get(function_name, None)
    if function:
        arguments = json.loads(arguments)
        results = function(**arguments)
    else:
        results = f"Error: function {function_name} does not exist"
    return results


@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions_with_functions(input, functions):
    MODEL = 'gpt-3.5-turbo'

    chat_completion = openai.chat.completions.create(
        model = MODEL,
        messages = input,
        functions = functions,
        function_call = "auto",  # Automatically call the function when needed
        seed = 2345)

    output = chat_completion.choices[0].message.content

    print("chat_completion.choices[0].message: ", chat_completion.choices[0].message)

    #if function called then call function, get details and then call chat completion once more with function o/p
    # print("chat_completion.choices[0].message.function_call", chat_completion.choices[0].message.function_call)
    if chat_completion.choices[0].message.function_call:
      # print("function called")

      function_name = chat_completion.choices[0].message.function_call.name
      arguments = chat_completion.choices[0].message.function_call.arguments
      # print("function_name. ", function_name)
      # print(". rguments :", arguments)
      function_response = execute_function_call(function_name, arguments)
      # print("function_response: ", function_response)

      input.append(chat_completion.choices[0].message)
      input.append({
          "role": "function",
          "name": function_name,
          "content": str(function_response),
      })
      output = get_chat_completions_with_functions(input, functions)

    return output


In [77]:

def get_order_info(user_input):
  """
  get order id from user - get_order_id
  once obtained - pull out order info from db/shopify API and make it json. - extract_order_info
  Based on what user asked prepare response - prepare_order_response
  """
  # print("inside get order info")
  tools = [
    {
        "name": "get_order_details",
        "description": "Gets the details of an order by order ID",
        "parameters": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "integer",
                    "description": "The ID of the order to retrieve"
                }
            },
            "required": ["order_id"]
        }
    }
  ]

  # first get order details
  conversation = get_order_initialisation(user_input)
  response_assistant = get_chat_completions_with_functions(conversation, functions=tools)
  conversation.append({"role": "assistant", "content": str(response_assistant)})

  print(response_assistant)
  user_input = ''
  while (response_assistant != 'exit') and (user_input != 'exit'):
    print("inside order info while loop")
    user_input = input("")  # Read user input
    moderation = moderation_check(user_input)  # Check moderation
    if moderation == 'Flagged':
      print("Sorry, this message has been flagged. Please restart your conversation.")
      break  # Exit the loop if the message is flagged
    conversation.append({"role": "user", "content": user_input})  # Append user input to conversation
    response_assistant = get_chat_completions_with_functions(conversation, functions=tools)  # Get assistant response
    print(response_assistant)  # Display assistant response
    conversation.append(({"role": "assistant", "content": response_assistant}))
  return "exit"



In [78]:
def get_order_initialisation(user_input):

  delimiter = "####"

  system_message = f"""
  You are an experienced customer support agent for an ecommerce direct to customer brand and your goal is to handle order related queries of a customer.
  Customer has been forwarded to you from a chat with an existing agent as user showed intent to know about their order details.
  You are expert in answering users order related queries.
   Do not say any greetings to user in begining as customer is coming from an existing chat and our aim will be provide a consistent experience.

  {delimiter}
  Users last message from prev chat was {user_input}

  {delimiter}

  The conversation flow involves three main thoughts. Follow these chain of thoughts and converse with the customer as customer support agent.
    - Thought 1: Obtaining the order id from user. First check if order id already exists in the users last message. If not then ask for it from user.
    - Thought 2: Once order id is obtained from user, then fetching details of the order from obtained order id using get_order_details.
    - Thought 3: Once we have order details from previous step, then find answer to user queries from the order details and reply back to customer with answer to their query.

  {delimiter}
  In case user asks questions which you do not know how to handle redirect user to manual customer support on support@support.com or call on 9383838338. Post that reply back with word "exit".

  {delimiter}

  If you are unable to find the details of order id shared by customer then tell them "Sorry, I am unable to find any order with order id <order id>. Please check and share order id again."

  {delimiter}
  Order information should always be accurate based on output received from get_order_details.


  {delimiter}
  Once you are done with helping user reply back with a single word "exit"

  """


  conversation = [{"role": "system", "content": system_message}]
  return conversation



In [79]:
def get_product_info_initialisation(last_message):
  delimiter = "####"
  system_message = f"""

  You are an experienced customer support agent for an ecommerce direct-to-customer brand, and your goal is to handle product-related queries of a customer.
  The customer has been forwarded to you from a chat with an existing agent as the user showed intent to know about products of the ecommerce company.
  You are an expert in answering users' product-related queries.
  Do not say any greetings to the user in the beginning as the customer is coming from an existing chat and our aim is to provide a consistent experience. Everytime you provide some answer ask your further question if needed else reply back with exit keyword.

  {delimiter}
  Users last message from previous chat was "{last_message}"
  {delimiter}

  The conversation flow involves three main thoughts. Follow these chain of thoughts and converse with the customer as a customer support agent:
    - Step 1: First evaluate what the user has asked in the last message and form a search query for it. If it is not clear to you what details the user is looking for, then ask further questions to clarify it and then form the search query.
    - Step 2: Fetch details of the product using search_product function.
    - Step 3: Based on output of search_product if any products found, answer user query else inform user that the product is not available.

  In case the user asks questions which you do not know how to handle, redirect the user to manual customer support at support@support.com or call 9383838338. Post that, reply back with the word "exit".

  {delimiter}
  Order information should always be accurate based on the output received from product details.

  {delimiter}
  Once you are done with helping the user, reply back with a single word "exit".
  """

  conversation = [{"role": "system", "content": system_message}]
  return conversation


def get_product_info(user_input):
  """
  get ecommerce site url from settings for example phool.co
  first check what user has asked. Form a seach query for it. create_search_query
  Then go to website of the ecommerce company and search for the product in it. perform_search
  then formulate the results and respond back - respond_to_user_query
  """
  tools = [
    {
        "name": "search_product",
        "description": "Search for product in product db",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "query to be searched for in product databased"
                }
            },
            "required": ["query"]
        }
    }
  ]

  conversation = get_product_info_initialisation(user_input)
  response_assistant = get_chat_completions(conversation)
  conversation.append({"role": "assistant", "content": str(response_assistant)})

  print(response_assistant)
  user_input = ''
  while (response_assistant != 'exit') and (user_input != 'exit'):
    # print("inside product info while loop")
    user_input = input("")  # Read user input
    moderation = moderation_check(user_input)  # Check moderation
    if moderation == 'Flagged':
      print("Sorry, this message has been flagged. Please restart your conversation.")
      break  # Exit the loop if the message is flagged
    conversation.append({"role": "user", "content": user_input})  # Append user input to conversation
    response_assistant = get_chat_completions_with_functions(conversation, functions=tools)  # Get assistant response
    conversation.append(({"role": "assistant", "content": response_assistant}))
    print(response_assistant)  # Display assistant response
  return "exit"


def get_return_policy_info():
  """
  get details of return and refund policy of company from urls of the pages in their websites in settings - prepare_refund_return_policy
  answer user query based on the return and refund policy - return_return_refund_policy_info
  """
  pass

def get_shipment_info():
  """
  get shipment awb no from user -
  """
  pass


In [80]:

INTENT_to_action_map = {
    "order_status": get_order_info,
    "product_info": get_product_info,
    "return_policy": get_return_policy_info,
    "shipping_info": get_shipment_info,
}


def chatbot():
  initial_input = input("Enter your query to start conversation with our support>>>>>\n ")
  user_input = ""
  conversation = initialize_conversation()
  is_intent_identified = False
  while(user_input != "exit" or response_assistant != "exit"):
    user_input = initial_input or input("")
    moderation = moderation_check(user_input)
    if moderation == 'Flagged':
      print("Sorry, this message has been flagged. Please restart your conversation.")
      initial_input = ""
      break
    conversation.append({"role": "user", "content": user_input})
    response_assistant = get_chat_completions(conversation)
    is_intent_identified = check_intent_identified(response_assistant)
    if is_intent_identified:
      action = INTENT_to_action_map.get(response_assistant)
      response_assistant = action(user_input)
    else:
      conversation.append(({"role": "assistant", "content": response_assistant}))
      print(str(response_assistant))
    initial_input = ""


In [ ]:
# can test with order id 19041580387436

chatbot()

Hello! I'm here to assist you. Can you please tell me if your query is about product information, order status, return policy, or shipment status?
What specific product information are you looking for? Please provide me with the name or description of the product you are interested in.
inside product info while loop
chat_completion.choices[0].message:  ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"query":"incense sticks"}', name='search_product'), tool_calls=None)
chat_completion.choices[0].message.function_call FunctionCall(arguments='{"query":"incense sticks"}', name='search_product')
function called
function_name.  search_product
. rguments : {"query":"incense sticks"}
function_response:  Title: Phool Badrinath Kesar Chandan Bambooless Incense Sticks, Price: 265, URL: https://phool.co/products/badrinath-kesar-chandan-bambooless-incense-sticks, Description: These bambooless incense sticks offer a pure, unadulterated aroma of kesar and c

In [ ]:
# {delimiter}
# once you find intent take action as follows:
# for check order details intent follow below steps:
#   1. first get order id from user
#   2. Once received get the order details
#   3. Prepare the answer to user query from the order details received.
# {delimiter}

# for product info intent:
#   1. get details of what info user is looking for
#   2. prepare search terms from user input
#   3. Search website {ECOM_WEBSITE} for the search term
#   4. if results found prepare response and respond back to user
#   5. if relevant results not found redirect user to manual support

# for refund or return policy questions:
#   1. Go to website {REFUND_URL} and get details of refund and return policy
#   2. check if answer to user query is present in refund and return policy
#   3. If present reply back to user with relevant details
#   4. if relevant results not found redirect user to manual support
